In [5]:
pip install google-api-python-client python-dotenv pandas


In [9]:
# jay_youtube_scraper.py
import os
import time
import pandas as pd
from dotenv import load_dotenv
from googleapiclient.discovery import build

# 讀取 API 金鑰
load_dotenv()
API_KEY = os.getenv("YOUTUBE_API_KEY")
youtube = build("youtube", "v3", developerKey=API_KEY)

# 取得播放清單中的所有影片
def get_videos_from_playlist(playlist_id):
    videos = []
    next_page_token = None

    while True:
        request = youtube.playlistItems().list(
            part="snippet",
            playlistId=playlist_id,
            maxResults=50,
            pageToken=next_page_token
        )
        response = request.execute()

        for item in response["items"]:
            snippet = item["snippet"]
            video_id = snippet["resourceId"]["videoId"]
            title = snippet["title"]
            videos.append({
                "video_id": video_id,
                "title": title
            })

        next_page_token = response.get("nextPageToken")
        if not next_page_token:
            break

    return pd.DataFrame(videos)

# 抓每部影片的留言
def get_comments(video_id, max_results=500):
    comments = []
    next_page_token = None
    count = 0

    while True:
        request = youtube.commentThreads().list(
            part="snippet",
            videoId=video_id,
            maxResults=100,
            pageToken=next_page_token,
            textFormat="plainText"
        )
        response = request.execute()

        for item in response["items"]:
            comment = item["snippet"]["topLevelComment"]["snippet"]
            comments.append({
                "video_id": video_id,
                "author": comment.get("authorDisplayName"),
                "text": comment.get("textDisplay"),
                "published_at": comment.get("publishedAt")
            })
            count += 1
            if count >= max_results:
                break

        next_page_token = response.get("nextPageToken")
        if not next_page_token or count >= max_results:
            break

        time.sleep(0.5)

    return pd.DataFrame(comments)

# 主流程：先抓清單 → 再批次抓留言
def scrape_jay_playlist(playlist_id, max_per_video=500, output_path="jay_playlist_comments.csv"):
    print("📥 抓取播放清單影片中...")
    video_df = get_videos_from_playlist(playlist_id)
    print(f"✅ 共找到 {len(video_df)} 部影片")

    all_comments = []

    for i, row in video_df.iterrows():
        vid = row["video_id"]
        title = row["title"]
        print(f"🎬 處理中：{title} ({vid})")

        try:
            df_comments = get_comments(vid, max_results=max_per_video)
            df_comments["video_title"] = title
            all_comments.append(df_comments)
        except Exception as e:
            print(f"⚠️ 無法處理影片 {vid}：{e}")

    # 合併並儲存
    all_df = pd.concat(all_comments, ignore_index=True)
    all_df.to_csv(output_path, index=False, encoding="utf-8-sig")
    print(f"✅ 所有留言已儲存至 {output_path}")

# 執行
if __name__ == "__main__":
    playlist_id = "PLoXw5hFmURCQOx8SZRrLhZUj8s3FOUWgZ"
    scrape_jay_playlist(playlist_id)

📥 抓取播放清單影片中...
✅ 共找到 165 部影片
🎬 處理中：周杰倫 Jay Chou【可愛女人 Cute Girl (feat.徐若瑄Vivian)】 -Official Music Video (87VUC4J_0Ps)
🎬 處理中：周杰倫 Jay Chou【完美主義 Perfectionism】-Official Music Video (gxZHH8x2zPg)
🎬 處理中：周杰倫 Jay Chou【星晴 Starry Mood】-Official Music Video (sTNJsIcPSvE)
🎬 處理中：周杰倫 Jay Chou【娘子 Wife】-Official Music Video (MpP9dw_j2PI)
🎬 處理中：周杰倫 Jay Chou【鬥牛 Basketball Match】-Official Music Video (npiAxeLtHDM)
🎬 處理中：周杰倫 Jay Chou【黑色幽默 Dark Humor】-Official Music Video (wRT-5heURhY)
🎬 處理中：周杰倫Jay Chou【伊斯坦堡 Istanbul】-Official Music Video (s9wvjSkR_Ms)
🎬 處理中：周杰倫 Jay Chou【印地安老斑鳩 Ancient Indian Turtledove】-Official Music Video (hnmWCmR6o7A)
🎬 處理中：周杰倫 Jay Chou【龍捲風 Tornado】-Official Music Video (RPWDeLqsN0g)
🎬 處理中：Jay Chou 周杰倫【反方向的鐘 Counter-clockwise Clock】-Official Music Video (fuM1aVCGR8c)
🎬 處理中：周杰倫 Jay Chou【愛在西元前 Love before AD】Official MV (5XK2C9w6oVk)
🎬 處理中：周杰倫 Jay Chou【爸 我回來了 Dad, I'm home】Official MV (nhyT8HDT4lg)
🎬 處理中：周杰倫 Jay Chou【簡單愛 Simple Love】Official MV (Y4xCVlyCvX4)
🎬 處理中：周杰倫 Jay Chou【忍者 Ninja】O